In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from ggplot import *

In [9]:
def get_df(history_file, n_hidden=[16, 32, 64, 128], cols=["n_hidden", "loss", "val_loss", "acc", "val_acc"]):
    dfs = []
    for n in n_hidden:
        with open(history_file.format(n), mode="br") as f:
            history = pickle.load(f)
            dfs.append(pd.DataFrame({
                    "n_hidden": n,
                    "loss": history["loss"],
                    "val_loss": history["val_loss"],
                    "acc": history["acc"],
                    "val_acc": history["val_acc"],
            }))

    history_seq = pd.concat(dfs, keys=[str(n) for n in n_hidden])
    history_seq = history_seq[cols]
    history_seq.n_hidden = history_seq.n_hidden.astype(np.int32).astype(str)
    return history_seq

In [10]:
# seq ... sequential model (GRU-GRU-Dense)
history_seq_file = "results/activity_model-117k-grid-150_smiles/activity-model-117k-70_30_train_test-150_smiles-history-{}_hidden.pickle"
history_seq = get_df(history_seq_file)
best_seq = pd.DataFrame()
for idx, df_select in history_seq.groupby(level=[0]):
    best_seq = best_seq.append(df_select.tail(1))
best_seq = best_seq.reset_index()
del best_seq["level_0"]
best_seq.columns.values[0] = "epoch"
best_seq

,epoch,n_hidden,loss,val_loss,acc,val_acc
0,29,16,0.313078,0.326423,0.857953,0.852975
1,29,32,0.267471,0.316537,0.884800,0.863042
2,14,64,0.253726,0.318738,0.892474,0.862669
3,10,128,0.223962,0.326513,0.907087,0.865635


In [12]:
# shared ... shared model ([GRU-GRU-GRU]-Dense)
history_shared_file = "results/activity_model-3_shared-117k-grid-150_smiles/activity_model-3_shared-117k-grid-150_smiles-history-{}_hidden.pickle"
history_shared = get_df(history_shared_file)
best_shared = pd.DataFrame()
for idx, df_select in history_shared.groupby(level=[0]):
    best_shared = best_shared.append(df_select.tail(1))
best_shared = best_shared.reset_index()
del best_shared["level_0"]
best_shared.columns.values[0] = "epoch"
best_shared

,epoch,n_hidden,loss,val_loss,acc,val_acc
0,29,16,0.343194,0.348635,0.841471,0.839336
1,29,32,0.314258,0.338042,0.859846,0.847976
2,20,64,0.280100,0.324888,0.878336,0.858701
3,14,128,0.254224,0.325103,0.890800,0.861288


In [5]:
#sns.set()
#sns.set_style("whitegrid")
#plot = sns.factorplot(x="loss", y="val_loss", data=history_seq, hue="n_hidden", scale=0.7, size=10, join=True)
#plot.set(xticks=np.arange(0, 1.1, 0.1))

In [6]:
"""
fig, ax = plt.subplots(1, 4, figsize=(8, 8))
g = ggplot(history_seq, aes(x="loss", y="val_loss", color="n_hidden")) + \
        geom_line(size=2.0) + \
        xlab("loss") + \
        ylab("val_loss") + \
        ggtitle("loss vs. validation loss | sequential model")
g
"""